In [ ]:
# Base setup: Colab secrets, repo clone, env vars
import os, pathlib
try:
    from google.colab import userdata  # type: ignore
    GH_TOKEN = userdata.get('GH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    AGNITRAAI_REPO = userdata.get('AGNITRAAI_REPO')
    if OPENAI_API_KEY: os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    if GOOGLE_API_KEY: os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    repo_path = pathlib.Path('/content/agnitraai')
    if pathlib.Path('/content').exists() and not repo_path.exists() and GH_TOKEN and AGNITRAAI_REPO:
        repo_url = f'https://{GH_TOKEN}@{AGNITRAAI_REPO}'
        get_ipython().system(f'git clone {repo_url} {repo_path}')
        os.chdir(repo_path)
    elif repo_path.exists():
        os.chdir(repo_path)
except Exception:
    pass


In [ ]:
# Install Agnitra locally so `agnitra` CLI is available in Colab
import pathlib, os
repo_path = pathlib.Path('/content/agnitraai')
if repo_path.exists():
    get_ipython().system('python -m pip install -q -e /content/agnitraai')
else:
    get_ipython().system('python -m pip install -q -e .')
# Quick sanity: show help (fallback to module if console script not found)
_ = get_ipython().system('agnitra --help >/dev/null 2>&1 || python -m agnitraai.cli.main --help >/dev/null 2>&1')
print('Agnitra CLI available.')


# Agnitra CLI Demo — TinyLlama

This notebook demonstrates a clean, CLI-first workflow:
- Prepare a tiny TinyLlama-like model
- Run profiling via `agnitra profile ...` (CPU/GPU auto-detected)
- Load telemetry artifacts and visualize key results


In [ ]:
# Minimal environment check and CLI availability
import shutil, subprocess, sys, json
try:
    import torch
    print(f'[Env] CUDA available: {torch.cuda.is_available()}')
    if torch.cuda.is_available():
        print('[Env] GPU:', torch.cuda.get_device_name(0))
    else:
        print('[Env] CPU-only runtime')
except Exception as e:
    print('[Env] Torch check skipped:', e)

if shutil.which('agnitra') is None:
    print('[Setup] Installing local CLI ...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-e', 'agnitraai'])
else:
    print('[Setup] agnitra CLI found')


## Prepare tiny model
Generates `tinyllama.pt` (TorchScript) and a small info JSON.

In [ ]:
import subprocess, sys
cmd = [sys.executable, 'prepare_tinyllama.py']
print('$', ' '.join(cmd))
subprocess.check_call(cmd)


## Profile using the Agnitra CLI
Runs the CLI and prints only the CLI's stdout for a clean view.

In [ ]:
import os, shlex, subprocess, shutil, sys
env = os.environ.copy()
env['TF_CPP_MIN_LOG_LEVEL'] = '3'
env['GYM_DISABLE_WARNINGS'] = '1'
base_cmd = 'agnitra profile tinyllama.pt --input-shape 1,16,64 --output telemetry.json'
if shutil.which('agnitra') is None:
    base_cmd = f'{sys.executable} -m cli.main profile tinyllama.pt --input-shape 1,16,64 --output telemetry.json'
print('$', base_cmd)
res = subprocess.run(shlex.split(base_cmd), env=env, check=True, capture_output=True, text=True)
print(res.stdout)  # keep CLI output clean (stderr suppressed)


## Load telemetry and summarize

In [ ]:
from pathlib import Path
import json
root = Path('agnitraai')/ 'context'
telemetry = json.loads(Path('telemetry.json').read_text())
train = json.loads(Path('telemetry_train.json').read_text()) if Path('telemetry_train.json').exists() else {'events': []}
summary = json.loads((root / 'profile_result_tinyllama.json').read_text())

def to_ms_ns(v):
    try: return float(v)/1e6
    except: return 0.0

total_cpu_ms = sum(to_ms_ns(e.get('cpu_time_total',0.0)) for e in telemetry.get('events', []))
total_cuda_ms = sum(to_ms_ns(e.get('cuda_time_total',0.0)) for e in telemetry.get('events', []))
print('[Summary] device:', summary.get('device'))
print('[Summary] events:', len(telemetry.get('events', [])))
print('[Summary] CPU total ms (inference):', round(total_cpu_ms, 3))
print('[Summary] CUDA total ms (inference):', round(total_cuda_ms, 3))
print('[Summary] training events:', len(train.get('events', [])))


## Visualize top events

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
ev = telemetry.get('events', [])
def top(evts, key, n=10):
    return sorted(evts, key=lambda e: e.get(key, 0.0), reverse=True)[:n]
# Prefer CPU on CPU-only runs
top_cpu = top(ev, 'cpu_time_total')
names = [e.get('name','') for e in top_cpu]
vals = [e.get('cpu_time_total',0.0)/1e6 for e in top_cpu]
plt.figure(figsize=(10,4))
plt.barh(names[::-1], vals[::-1])
plt.xlabel('CPU time (ms)')
plt.title('Top ops by CPU time (inference)')
plt.tight_layout(); plt.show()


## Artifact paths

In [ ]:
from pathlib import Path
out = [
    'telemetry.json',
    'telemetry_train.json',
    'agnitraai/context/layer_log_tinyllama.json',
    'agnitraai/context/profile_result_tinyllama.json',
]
for p in out:
    print('[Artifact]', p, 'exists:', Path(p).exists())


## Interactive CLI playground

Run Agnitra commands without leaving the notebook.

1. Pick a preset (or type your own command).
2. Press **Run CLI** to execute it in this environment.
3. Inspect stdout, stderr, and the exit code right below.

Tip: Start with the profile preset to generate `telemetry.json`, which powers the IR views that follow.

In [ ]:
import os
import shlex
import subprocess
import sys
from pathlib import Path

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError:
    print("ipywidgets is required for the interactive CLI demo.")
    print("Install it via `pip install ipywidgets` and rerun this cell.")
else:
    cwd = Path.cwd()
    profile_target = cwd / "tinyllama.pt"
    profile_arg = shlex.quote(str(profile_target)) if profile_target.exists() else "tinyllama.pt"

    presets = {
        "Profile tinyllama (python -m cli.main)": (
            f"python -m cli.main profile {profile_arg} --input-shape 1,16,64 "
            "--output telemetry_interactive.json"
        ),
        "Optimize demo model": "python -m cli.optimize --model demo-model",
        "Show agnitra help": "agnitra --help",
    }

    preset_keys = list(presets.keys())
    default_status = f"<span style='color:#616161;'>Ready · cwd: {cwd}</span>"

    preset = widgets.Dropdown(
        options=preset_keys,
        description="Preset",
        layout=widgets.Layout(width="50%"),
    )
    run_button = widgets.Button(description="Run CLI", icon="play", button_style="success")
    clear_button = widgets.Button(description="Clear", icon="trash", layout=widgets.Layout(width="100px"))
    status = widgets.HTML(value=default_status)
    command = widgets.Textarea(
        value=presets[preset_keys[0]],
        description="Command",
        layout=widgets.Layout(width="100%", height="80px"),
        continuous_update=False,
    )
    output = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="8px"))

    def _apply_preset(change):
        new_value = change.get("new")
        if new_value in presets:
            command.value = presets[new_value]

    def _render_status(message: str, color: str = "#616161") -> None:
        status.value = f"<span style='color:{color};'>{message}</span>"

    def _run_command(_):
        text = command.value.strip()
        if not text:
            _render_status("Enter a command to run.", color="#c62828")
            return
        try:
            args = shlex.split(text)
        except ValueError as err:
            _render_status(f"Parse error: {err}", color="#c62828")
            return

        run_button.disabled = True
        run_button.icon = "hourglass-half"
        run_button.button_style = "warning"
        _render_status("Running...", color="#0277bd")
        output.clear_output()

        try:
            result = subprocess.run(args, text=True, capture_output=True, cwd=str(cwd))
        except FileNotFoundError as err:
            _render_status(str(err), color="#c62828")
        except Exception as exc:
            _render_status(f"Failed: {exc}", color="#c62828")
        else:
            with output:
                print("$", text)
                if result.stdout:
                    print(result.stdout)
                if result.stderr:
                    print(result.stderr, file=sys.stderr)
                print(f"[exit code] {result.returncode}")
                artifact_names = [
                    "telemetry_interactive.json",
                    "telemetry.json",
                    "telemetry_train.json",
                    "telemetry_val.json",
                    "graph_ir.json",
                ]
                produced = []
                for name in artifact_names:
                    candidate = cwd / name
                    if candidate.exists():
                        try:
                            produced.append(candidate.relative_to(cwd))
                        except ValueError:
                            produced.append(candidate)
                if produced:
                    print("Artifacts:")
                    for path in produced:
                        print(f"  • {path}")
            color = "#2e7d32" if result.returncode == 0 else "#c62828"
            status_text = "Finished"
            if result.returncode == 0:
                status_text += " · exit code 0"
            else:
                status_text += f" · exit code {result.returncode}"
            _render_status(status_text, color=color)
        finally:
            run_button.disabled = False
            run_button.icon = "play"
            run_button.button_style = "success"

    def _clear_output(_):
        output.clear_output()
        status.value = default_status

    preset.observe(_apply_preset, names="value")
    preset.value = preset_keys[0]
    _apply_preset({"new": preset.value})
    run_button.on_click(_run_command)
    clear_button.on_click(_clear_output)

    display(
        widgets.VBox(
            [
                widgets.HBox([preset, run_button, clear_button]),
                command,
                status,
                output,
            ]
        )
    )


## Inspect Graph IR

Turn the TinyLlama demo into an FX-based intermediate representation enriched with telemetry from the CLI runs above. If telemetry is missing, the extractor still works (validation falls back gracefully).

In [ ]:
import json
from pathlib import Path

import torch

from agnitra.core.ir.graph_extractor import extract_graph_ir
from prepare_tinyllama import TinyLlama

telemetry_candidates = [
    Path("telemetry_interactive.json"),
    Path("telemetry.json"),
]
telemetry_payload = None
for candidate in telemetry_candidates:
    if candidate.exists():
        telemetry_payload = json.loads(candidate.read_text())
        print(f"Loaded telemetry from {candidate}.")
        break
else:
    print("No telemetry file found; run the profiling CLI preset above to gather telemetry overlays.")

model = TinyLlama().eval()
example_inputs = (torch.randn(1, 16, 64),)

try:
    graph_ir = extract_graph_ir(
        model,
        example_inputs=example_inputs,
        telemetry=telemetry_payload,
        validate=True,
    )
except ValueError as exc:
    print(f"Validation failed: {exc}")
    print("Retrying with validate=False so you can still inspect the graph.")
    graph_ir = extract_graph_ir(
        model,
        example_inputs=example_inputs,
        telemetry=telemetry_payload,
        validate=False,
    )

node_count = len(graph_ir)
with_cuda = sum(1 for node in graph_ir if node.get("cuda_time_ms") is not None)
with_cpu = sum(1 for node in graph_ir if node.get("cpu_time_ms") is not None)
print(f"Extracted {node_count} IR nodes.")
print(f"  • {with_cuda} nodes carry CUDA timing data")
print(f"  • {with_cpu} nodes carry CPU timing data")

ir_path = Path("graph_ir.json")
ir_path.write_text(json.dumps(graph_ir, indent=2))
print(f"Saved a snapshot to {ir_path.resolve()}")

graph_ir[:3]


### Interactive IR explorer

Filter, search, and visualise the IR nodes. Change the metric, narrow by FX node kind, or focus on specific telemetry sources to highlight hotspots.

In [ ]:
import pandas as pd
from IPython.display import Markdown, display

if "graph_ir" not in globals():
    raise RuntimeError("Run the IR extraction cell above first.")

if not graph_ir:
    raise ValueError("Graph IR is empty; run the profiling flow and extraction again.")

df_ir = pd.DataFrame(graph_ir)

if "telemetry_sources" not in df_ir:
    df_ir["telemetry_sources"] = [[] for _ in range(len(df_ir))]
else:
    df_ir["telemetry_sources"] = df_ir["telemetry_sources"].apply(
        lambda items: items if isinstance(items, list) else []
    )

numeric_cols = [
    "cuda_time_ms",
    "cpu_time_ms",
    "memory_bytes",
]
for col in numeric_cols:
    if col in df_ir:
        df_ir[col] = pd.to_numeric(df_ir[col], errors="coerce")

summary_lines = [
    f"Total nodes: {len(df_ir)}",
    f"call_module nodes: {int((df_ir['kind'] == 'call_module').sum())}",
    f"Nodes with telemetry: {int(df_ir['telemetry_sources'].apply(bool).sum())}",
]

summary_md = "**Quick stats**\n\n" + "\n".join(f"- {line}" for line in summary_lines)
display(Markdown(summary_md))

try:
    import ipywidgets as widgets
    import matplotlib.pyplot as plt
except ImportError:
    print("ipywidgets and matplotlib are required for the interactive view. Install via `pip install ipywidgets matplotlib`.")
else:
    try:
        import plotly.express as px  # type: ignore
    except ImportError:  # Plotly is optional
        px = None
        print("Plotly not installed; falling back to static Matplotlib charts. Install via `pip install plotly` for interactivity.")

    metric_options = [
        ("CUDA time (ms)", "cuda_time_ms"),
        ("CPU time (ms)", "cpu_time_ms"),
        ("Memory bytes", "memory_bytes"),
    ]
    metric_lookup = {value: label for label, value in metric_options}
    metric_dropdown = widgets.Dropdown(options=metric_options, value="cuda_time_ms", description="Metric")

    max_top = max(1, min(30, len(df_ir))) if len(df_ir) else 10
    min_top = 1 if max_top < 3 else 3
    default_top = min(10, max_top)
    top_n_slider = widgets.IntSlider(
        value=max(default_top, min_top),
        min=min_top,
        max=max_top,
        step=1,
        description="Top N",
    )

    kinds = sorted(k for k in df_ir["kind"].dropna().unique())
    telemetry_tags = sorted({tag for tags in df_ir["telemetry_sources"] for tag in (tags or [])})

    kind_selector = widgets.SelectMultiple(
        options=kinds,
        description="Kinds",
        layout=widgets.Layout(width="200px", height="180px"),
    )
    telemetry_selector = widgets.SelectMultiple(
        options=telemetry_tags,
        description="Telemetry",
        layout=widgets.Layout(width="200px", height="180px"),
    )
    name_filter = widgets.Text(placeholder="Substring filter (name/op)", description="Search")
    output_table = widgets.Output()

    def _filter_frame():
        frame = df_ir.copy()
        if kind_selector.value:
            frame = frame[frame["kind"].isin(kind_selector.value)]
        if telemetry_selector.value:
            selected = set(telemetry_selector.value)
            frame = frame[
                frame["telemetry_sources"].apply(lambda items: bool(selected.intersection(items)))
            ]
        text = name_filter.value.strip()
        if text:
            frame = frame[
                frame["name"].str.contains(text, case=False, na=False)
                | frame["op"].str.contains(text, case=False, na=False)
            ]
        return frame

    def _render(_=None):
        frame = _filter_frame()
        metric = metric_dropdown.value
        output_table.clear_output()
        with output_table:
            if metric not in frame or frame.empty:
                print("No nodes match the current filter.")
                return
            subset = frame[["name", "op", metric, "kind", "telemetry_sources"]].dropna(subset=[metric])
            subset = subset.sort_values(metric, ascending=False).head(top_n_slider.value)
            if subset.empty:
                print("No numeric data available for the chosen metric.")
                return
            display(subset.reset_index(drop=True))
            if px is not None:
                fig = px.bar(
                    subset,
                    x=metric,
                    y="name",
                    orientation="h",
                    color="kind",
                    hover_data=["op", "telemetry_sources"],
                    title="Top IR nodes",
                )
                fig.update_layout(height=420, margin=dict(l=0, r=0, t=60, b=0))
                fig.show()
            else:
                fig, ax = plt.subplots(figsize=(10, 4))
                ax.barh(subset["name"][::-1], subset[metric][::-1], color="#1976d2")
                ax.set_xlabel(metric_lookup.get(metric, metric))
                ax.set_ylabel("Node")
                ax.set_title("Top IR nodes")
                plt.tight_layout()
                plt.show()

    metric_dropdown.observe(_render, names="value")
    kind_selector.observe(_render, names="value")
    telemetry_selector.observe(_render, names="value")
    name_filter.observe(_render, names="value")
    top_n_slider.observe(_render, names="value")

    controls = widgets.HBox(
        [
            widgets.VBox([metric_dropdown, top_n_slider, name_filter]),
            widgets.VBox([kind_selector, telemetry_selector]),
        ]
    )
    display(widgets.VBox([controls, output_table]))
    _render()
